# Pushing property information to the Data Skeptic Home Sales Project API

Writing that title out makes me realize how poorly this project is named!  Perhaps some volunteer might take up the challenge of branding these efforts...

In any event, I wanted to write up a quick blog post giving people a BARE BONES example, using Python, of how to push property data to the API.  The objective of this demo is to be accessible to anyone that can get Python running.  In time, a more formal module should be defined, but for now, let's keep it light weight.

In [34]:
import requests
from datetime import datetime
import json

Right now the API is hosted on Heroku at the base URL shown below.  In time, this might be moved.  api.dataskeptic.com might be a better TLD, so I've called this out as a parameter.

In [33]:
baseurl = 'https://home-sales-data-api.herokuapp.com/'

The requests package is great for calling straightforward RESTful APIs.  Since the request below is a simple GET request, you could make it in your browser if you'd like.  Let's ask the API for all listings.

In [35]:
r = requests.get(baseurl + '/api/property')

In [38]:
response = json.loads(r.content)
print(response)

[{u'geocoded_address': u'123 Main St, Schenevus, NY 12155, USA', u'bathrooms': 2.0, u'features': [], u'upload_timestamp': u'2016-04-05T02:46:50.380670Z', u'listing_timestamp': u'2016-03-31T21:09:00Z', u'building_size': 1400.0, u'price': 123456.0, u'bedrooms': 3.0, u'size_units': u'M', u'car_spaces': 1.0, u'raw_address': u'123 Main St.', u'listing_type': u'F', u'land_size': 2000.0, u'id': 1}, {u'geocoded_address': u'123 Test Rd, Richmond, IN 47374, USA', u'bathrooms': 0.0, u'features': [], u'upload_timestamp': u'2016-04-05T02:57:20.785874Z', u'listing_timestamp': u'2016-04-05T02:57:20.059246Z', u'building_size': 10.0, u'price': 1.0, u'bedrooms': 0.0, u'size_units': u'M', u'car_spaces': None, u'raw_address': u'123 Test Ave', u'listing_type': u'F', u'land_size': 20.0, u'id': 2}]


Clearly, I'm writing this blog post in early days of the API as I only found a few listings, all of which appear to be tests.  But that's ok, my next post will be about getting some actual data into the API so I can do more interesting things.

As a next step, let's push in one more test property, so you can see how the upload happens.

In [41]:
# Defining a few variables explicitly

price = 1
bedrooms = 0
bathrooms = 0
car_spaces = None
building_size = 10
land_size = 20
size_units = 'M' # metric
raw_address = '123 Test Ave'

# This is the actual request we want to send in JSON format to the server.
request = {
    "listing_timestamp": str(datetime.now()),
    "listing_type": 'F', # for sale
    "price": price,
    "bedrooms": bedrooms,
    "bathrooms": bathrooms,
    "car_spaces": car_spaces,
    "building_size": building_size,
    "land_size": land_size,
    "size_units": size_units,
    "raw_address": raw_address,
    "features": []
}

In [42]:
# Let's send that request to the API, and see what response we get back
r = requests.post(baseurl + '/api/property/', data = request)
print(r.content)

{"id":3,"upload_timestamp":"2016-04-05T03:08:21.202776Z","listing_timestamp":"2016-04-05T03:08:09.140107Z","listing_type":"F","price":1.0,"bedrooms":0.0,"bathrooms":0.0,"car_spaces":null,"building_size":10.0,"land_size":20.0,"size_units":"M","raw_address":"123 Test Ave","geocoded_address":"123 Test Rd, Richmond, IN 47374, USA","features":[]}


Looks like our upload was successful.  Let's request all listings and check that the one with this ID is there and matches our address.

In [43]:
r = requests.get(baseurl + '/api/property')
resp = json.loads(r.content)

In [46]:
success = False
for prop in resp:
    if prop['id']==3:
        if prop['raw_address']==raw_address:
            success=True
print success

True


That's literally it to submit a new listing.  I did not get into properties (like having a pool), but my goal of presenting a very basic MVP for others to follow seems to be complete here.

My vision is that independent contributors can start finding data sources and pushing data to the API.  As their implementations mature, I will set up a formal process for sending pull requests to merge their code into a repository that tracks all sufficiently developed integrations.  Those integrations can then be run centrally, but maintained independently by the community via pull requests.